# Задание 10.

Вам предоставлен набор данных по качеству воздуха в китайских городах в виде csv файлов.
Основная метрика, которая показывает качество воздуха - PM25(<a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4740125/'>подробнее о PM25</a>), указана в колонках, которые начинаются на PM_.
Остальные столбцы содержат либо данные о времени, в которое были произведены измерения, либо данные о погоде в этот момент. DEWP - точка росы, HUMI - влажность, PRES - давление, TEMP - температура, cbwd - направление ветра, Iws - его скорость, precipation - осадки, Iprec - накопленные осадки.

Для выполнения задания необходимо ответить на вопросы по данным, заполнив ячейки ноутбука. Сам ноутбук необходимо выложить в виде отдельного репозитория на github. Ссылку для клонирования репозитория необходимо приложить в решение задания.

## 1. Загрузите в виде DataFrame набор данных по качеству воздуха Пекина
Ссылка на набор данных: https://disk.yandex.ru/d/1UKJoV1ULW1NLQ

In [1]:
import pandas as pd
df = pd.read_csv('BeijingPM20100101_20151231.csv')


## 2. Сколько всего строк находится в наборе данных?

In [3]:
print(df.shape[0])


52584


## 3. Сколько измерений уровня загрязнения воздуха находится в наборе данных?

In [4]:
print(df.No.count())


52584


## 4. Введите столбец среднего качества воздуха PM_mean с усреднением показаний со всех станций слежения.

In [22]:
df_PM_mean = pd.DataFrame({'Dongsi': df.PM_Dongsi,
                      'Dongsihuan': df.PM_Dongsihuan,
                      'Nongzhanguan': df.PM_Nongzhanguan,
                      }).mean(axis=1)
print('PM_mean')
print(df_PM_mean)


PM_mean
0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
52579    139.666667
52580    168.333333
52581    199.333333
52582    222.333333
52583           NaN
Length: 52584, dtype: float64


## 5. Какое направление ветра преобладает в Пекине?

In [6]:
print(df.cbwd.mode()[0])


SE


## 6. В какой день качество воздуха является наихудшим(самая большая концентрация частиц)? Вывести дату в типе данных datetime стандартной библиотеки Python.

In [35]:
str_max = df_PM_mean[df_PM_mean == df_PM_mean.max()]
index_max = str_max.index[0]
df_date_max = pd.DataFrame({
    'year': [df.year[index_max]],
    'month': [df.month[index_max]],
    'day': [df.day[index_max]],
    'hour': [df.hour[index_max]]
})
print(pd.to_datetime(df_date_max)[0])


2014-02-14 23:00:00


## 7. Какой из сезонов в Пекине в среднем наименее неблагоприятный для легких за последние 2 года наблюдений?

In [127]:
df_season_2lastyear_mean = df.where(df.year.max() < df.year + 2).groupby('season').mean()
season_number = df_season_2lastyear_mean.where(df_season_2lastyear_mean.PM_Dongsi + df_season_2lastyear_mean.PM_Dongsihuan + df_season_2lastyear_mean.PM_Nongzhanguan ==
                               (df_season_2lastyear_mean.PM_Dongsi + df_season_2lastyear_mean.PM_Dongsihuan + df_season_2lastyear_mean.PM_Nongzhanguan).max()).sort_values('No').index[0]
print(season_number)

4.0


## 8. Коррелирует ли влажность с качеством воздуха?
Можно воспользоваться корреляцией Пирсона из numpy, а также встроенными функциями Pandas.

In [32]:
# Чем ближе к 1, тем сильнее положительная корреляция.
print(df['HUMI'].corr(df_PM_mean))


0.39000021743100094


## 9. Какой из сезонов в Пекине самый дождливый(в среднем по накоплению осадков)?

In [162]:
df_Iprec_season_mean_sort = df.groupby(
    'season').mean().sort_values('Iprec', ascending=False)
print(df_Iprec_season_mean_sort.Iprec.index[0])


3


## 10. Становится ли качество воздуха во время дождя лучше, чем оно было сразу перед дождем?

In [13]:
# кол-во значений когда качество воздуха лучше во время дождя(есть осадки)
during_rain = df.where((df.precipitation > 0) & (df.precipitation.shift(-1) == 0) & (df.PM_Dongsi.shift(-1) + df.PM_Dongsihuan.shift(-1) + df.PM_Nongzhanguan.shift(-1) <
         df.PM_Dongsi + df.PM_Dongsihuan + df.PM_Nongzhanguan))['No'].count()
# кол-во значений когда качество воздуха лучше до дождя(нет осадков)
before_rain = df.where((df.precipitation > 0) & (df.precipitation.shift(-1) == 0) & (df.PM_Dongsi.shift(-1) + df.PM_Dongsihuan.shift(-1) + df.PM_Nongzhanguan.shift(-1) >
         df.PM_Dongsi + df.PM_Dongsihuan + df.PM_Nongzhanguan))['No'].count()
print(before_rain < during_rain)

False
